In [29]:
import numpy as np
import pandas as pd

In [30]:
df = pd.read_csv('C:/users/LL/Documents/GitHub/AMLF_projects/data.csv')

In [31]:
df = df.drop(columns = ['Unnamed: 0'])

In [32]:
df['r(t+1)'] = df.groupby('permno')['return'].shift(-1)

### handle missing data

According to note 30 (bottom of p 2248): "Another issue is missing characteristics, which we replace with the cross-sectional median at each month for each stock, respectively." <br>
Hence, calculate monthly cross-sectional median for features: **'mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr'**.

In [23]:
# df['Date'] = pd.to_datetime(df['Date'])
# df.set_index('Date', inplace=True)

In [27]:
df_filled = df.copy()
for feature in ['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']:
    df_filled[feature] = df_filled.groupby('Timestamp')[feature].transform(lambda x: x.fillna(x.median()))

,permno,return,mom1m,mom12m,chmom,indmom,mom36m,turn,mvel1,dolvol,...,retvol,idiovol,beta,betasq,ep,sp,agr,nincr,return(t-1),r(t+1)
Date,,,,,,,,,,,,,,,,,,,,,
2001-01-31,13610,-0.202242,0.277978,-0.082232,0.518490,0.198455,-0.258322,0.820391,9.897840e+05,12.968131,...,0.032888,0.056769,0.398706,0.158967,0.019041,1.472909,0.325935,5.0,NaN,0.161318
2001-01-31,13856,-0.112756,0.095372,0.300233,-0.147620,0.069669,0.000718,0.660636,7.152231e+07,17.527472,...,0.022389,0.042020,0.106574,0.011358,0.039970,0.397105,0.225463,1.0,NaN,0.046940
2001-01-31,13901,0.010566,0.166088,0.759861,0.504130,0.400659,-0.440929,0.775189,9.783550e+07,17.773933,...,0.023475,0.050243,0.088382,0.007811,0.142695,1.148088,-0.024383,2.0,NaN,0.094426
2001-01-31,13928,0.033114,0.006637,0.236486,0.039925,0.400659,0.025686,0.813903,1.451888e+07,16.111030,...,0.023917,0.037427,0.159983,0.025595,0.051091,1.138525,-0.069288,1.0,NaN,-0.084292
2001-01-31,13936,0.014335,0.009709,0.574141,0.224657,-0.075486,-0.397110,0.755288,3.550430e+05,12.518458,...,0.030343,0.068491,1.060490,1.124640,0.091598,6.902488,0.000838,0.0,NaN,0.149662


In [25]:
df_filled = df.loc[:,
                   ['permno', 'mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol',
                    'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']].fillna(df.groupby([pd.Grouper(freq='M'), 'permno']).transform('median'))



In [26]:
df_filled.isna().sum()

permno        0
mom1m         3
mom12m      269
chmom       269
mom36m     1566
turn          9
dolvol        6
idiovol     352
beta        352
betasq      352
ep          473
sp          473
agr        1026
nincr      2794
dtype: int64

In [ ]:
df.loc[:, ['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']] = df_filled.loc[:,['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']]



In [14]:
df.isna().sum()

permno            0
return            0
mom1m             3
mom12m          269
chmom           269
indmom            0
mom36m         1566
turn              9
mvel1             0
dolvol            6
ill               0
zerotrade         0
baspread          0
retvol            0
idiovol         352
beta            352
betasq          352
ep              473
sp              473
agr            1026
nincr          2794
return(t-1)     500
r(t+1)          500
dtype: int64

### split data
**split training, validation, and testing datasets**

training : validation : testing = 6 yr : 4yr : 9 yr <br>
Also drop the first and last month due to the absence of r(t+1) and return(t-1)

In [13]:
training = df[:'2007-01-01'].dropna()
validation = df['2007-01-02':'2011-01-01']
testing = df[:'2020-01-01'].dropna()

**separate X and y**

In [ ]:
X_train = training.drop(columns = ['permno', 'return', 'r(t+1)'])
y_train = training['r(t+1)']

In [ ]:
X_val = validation.drop(columns = ['permno', 'return', 'r(t+1)'])
y_val = validation['r(t+1)']

In [ ]:
X_test = testing.drop(columns = ['permno', 'return', 'r(t+1)'])
y_test = testing['r(t+1)']

In [ ]:
print(X_train.isna().values.sum())
print(X_val.isna().values.sum())
print(X_test.isna().values.sum())
print(y_train.isna().values.sum())
print(y_val.isna().values.sum())
print(y_test.isna().values.sum())